# Выбор локации для скважины

Мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно постоить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль, проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression

from sklearn.utils import shuffle


from sklearn.metrics import (precision_recall_curve,
                             mean_absolute_error,
                             confusion_matrix,
                             recall_score,
                             precision_score,
                             f1_score,
                             roc_auc_score,
                             mean_squared_error)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler


In [2]:
data_0= pd.read_csv('/datasets/geo_data_0.csv')
data_1= pd.read_csv('/datasets/geo_data_1.csv')
data_2= pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [5]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [7]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [9]:
display(data_0.duplicated().sum())
display(data_1.duplicated().sum())
display(data_2.duplicated().sum())

0

0

0

### Вывод

Каждая таблица имеет 10000 строк и 5 столбцов

Из документации:

* `id` — уникальный идентификатор скважины;
* `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` — объём запасов в скважине (тыс. баррелей).

Данные таблицы не имеют дубликатов, пропусков и явных анамальных значений

Данные таблицы имеют правильный регистр и тип данных

Можем приступать к обучению модели

## Обучение и проверка модели

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

In [10]:
def train_valid(data):
    global features, target, features_train, features_valid, target_train, target_valid
    features = data.drop(['product', 'id'], axis=1)
    target = data['product']
    #обучающая & валидационная
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    display(features_train.shape) #обучающая
    display(target_valid.shape) #валидационная
train_valid(data_0)

(75000, 3)

(25000,)

In [11]:
model = LinearRegression()
model.fit(features_train, target_train)

predictions_0 = model.predict(features_valid)
mean_predictions_0 = predictions_0.mean()

rmse_0 = np.sqrt(mean_squared_error(target_valid, predictions_0))

target_valid_0 = target_valid
mean_valid_0 = target_valid_0.mean()
print('mean valid:', mean_valid_0)

print('mean predictions:', mean_predictions_0)
print('rmse:', rmse_0)

mean valid: 92.07859674082927
mean predictions: 92.59256778438035
rmse: 37.5794217150813


In [12]:
train_valid(data_1)

(75000, 3)

(25000,)

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)

predictions_1= model.predict(features_valid)

mean_predictions_1 = predictions_1.mean()
rmse_1 = np.sqrt(mean_squared_error(target_valid, predictions_1))

target_valid_1 = target_valid
mean_valid_1 = target_valid_1.mean()
print('mean valid:', mean_valid_1)

print('mean:', mean_predictions_1)
print('rmse:', rmse_1)

mean valid: 68.72313602435997
mean: 68.728546895446
rmse: 0.893099286775617


In [14]:
train_valid(data_2)

(75000, 3)

(25000,)

In [15]:
model = LinearRegression()
model.fit(features_train, target_train)

predictions_2 = model.predict(features_valid)

mean_predictions_2 = predictions_2.mean()
rmse_2 = np.sqrt(mean_squared_error(target_valid, predictions_2))

target_valid_2 = target_valid
mean_valid_2 = target_valid_2.mean()
print('mean valid:', mean_valid_2)

print('mean:', mean_predictions_2)
print('rmse:', rmse_2)

mean valid: 94.88423280885438
mean: 94.96504596800489
rmse: 40.02970873393434


In [16]:
mean_rmse_frame= pd.DataFrame()
mean_rmse_frame['mean_valid']= mean_valid_0, mean_valid_1, mean_valid_2
mean_rmse_frame['mean_predictions']= mean_predictions_0, mean_predictions_1, mean_predictions_2
mean_rmse_frame['rmse']= rmse_0, rmse_1, rmse_2

display(mean_rmse_frame)

,mean_valid,mean_predictions,rmse
0,92.078597,92.592568,37.579422
1,68.723136,68.728547,0.893099
2,94.884233,94.965046,40.029709


### Вывод

Средние на валидационной выборке очень похожи на предсказанные средние, вполне вероятно, что модели хорошо обучены

Но здесь очень заметна разница между второй моделью(data_1) и осталными(0 и 2)

Это логично, ведь, если обратить внимание на этап подготовки данных, мы увидим, что таблица `data_1` имеет намного больший разброс параметров, нежели оставшиеся таблицы

Можем заранее предположить, что так как данные региона 1 очень отличаются, скорее всего, это будет либо единственный реггион который нам подходит, либо наоборот - единственный регион, который нам не подходит

## Подготовка к расчёту прибыли

* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью

In [17]:
def reset_indx(target_valid):
    target_valid = target_valid.reset_index()
    target_valid = target_valid.drop(['index'], axis=1)
    return target_valid
target_valid_0= reset_indx(target_valid_0)
target_valid_1= reset_indx(target_valid_1)
target_valid_2= reset_indx(target_valid_2)

In [18]:
key_values= pd.DataFrame()
key_values['predictions_0']= predictions_0
key_values['target_valid_0']= target_valid_0
key_values['predictions_1']= predictions_1
key_values['target_valid_1']= target_valid_1
key_values['predictions_2']= predictions_2
key_values['target_valid_2']= target_valid_2
key_values['predictions_0']= predictions_0

display(key_values)

,predictions_0,target_valid_0,predictions_1,target_valid_1,predictions_2,target_valid_2
0,95.894952,10.038645,82.663314,80.859783,93.599633,61.212375
1,77.572583,114.551489,54.431786,53.906522,75.105159,41.850118
2,77.892640,132.603635,29.748760,30.132364,90.066809,57.776581
3,90.175134,169.072125,53.552133,53.906522,105.162375,100.053761
4,70.510088,122.325180,1.243856,0.000000,115.303310,109.897122
...,...,...,...,...,...,...
24995,103.037104,170.116726,136.869211,137.945408,78.765887,28.492402
24996,85.403255,93.632175,110.693465,110.992147,95.603394,21.431303
24997,61.509833,127.352259,137.879341,137.945408,99.407281,125.487229
24998,118.180397,99.782700,83.761966,84.038886,77.779912,99.422903


In [19]:
a = 10_000_000_000/200 # максимальная стоимость одной разрабатываемой скважины
b= a/450_000 #минимальное среднее количество продукта в тысячах баррелей в месторождениях региона, достаточное для разработки
b

111.11111111111111

In [ ]:
def prof(target, predictions, count):
    predict_sorted = predictions.sort_values(ascending=False)
    selected = target[predict_sorted.index][:count]
    return  selected.sum()*450_000-10**10

### Вывод


Для дальнейших расчётов мы создали таблицу в которой каждому предсказанию соответствует значение валидационной выборки

Также созжали функцию для расчёта прибыли

## Расчёт прибыли и рисков 

In [21]:
state = np.random.RandomState(12345)
values = []
def bootstep(target, probabilities):
    for i in range(1000):
        target_subsample= target.sample(n=500,replace=True,random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        rev = prof(target_subsample,probs_subsample,200)
        values.append(rev)
    return values

In [26]:
bootstep_0 = bootstep( key_values['target_valid_0'], key_values['predictions_0'])
bootstep_0 = pd.Series(bootstep_0)
lower = bootstep_0.quantile(0.025)
apper = bootstep_0.quantile(0.975)

mean= bootstep_0.mean()

In [27]:
print('регион 0')
print(f'Средняя прибыль: {mean} рублей')
print('Риск убытков:', (len(bootstep_0[bootstep_0<0])/len(bootstep_0))*100, '%')
print(f'С вероятностью 95% выручка будет не менее: {lower} рублей и не более: {apper}')


регион 0
Средняя прибыль: 477723131.79467154 рублей
Риск убытков: 2.65 %
С вероятностью 95% выручка будет не менее: -6930525.967865173 рублей и не более: 976440320.83285


In [24]:
values = []
bootstep_1 = bootstep( key_values['target_valid_1'], key_values['predictions_1'])
bootstep_1 = pd.Series(bootstep_1)
lower = bootstep_1.quantile(0.025)
apper = bootstep_1.quantile(0.975)
mean= bootstep_1.mean()

In [25]:
print('регион 1')
print(f'Средняя прибыль: {mean} рублей')
print('Риск убытков:', (len(bootstep_1[bootstep_1<0])/len(bootstep_1))*100, '%')
print(f'С вероятностью 95% выручка будет не менее: {lower} рублей и не более: {apper}')


регион 1
Средняя прибыль: 518259493.6973249 рублей
Риск убытков: 0.3 %
С вероятностью 95% выручка будет не менее: 128123231.43308444 рублей и не более: 953612982.0669085


In [28]:
values = []
bootstep_2 = bootstep( key_values['target_valid_2'], key_values['predictions_2'])
bootstep_2 = pd.Series(bootstep_2)
lower = bootstep_2.quantile(0.025)
apper = bootstep_2.quantile(0.975)
mean= bootstep_2.mean()

In [29]:
print('регион 2')
print(f'Средняя прибыль: {mean} рублей')
print('Риск убытков:', (len(bootstep_2[bootstep_2<0])/len(bootstep_2))*100, '%')
print(f'С вероятностью 95% выручка будет не менее: {lower} рублей и не более: {apper}')


регион 2
Средняя прибыль: 424212488.0991597 рублей
Риск убытков: 4.9 %
С вероятностью 95% выручка будет не менее: -94920268.4330111 рублей и не более: 932890021.6850643


## Вывод

Техникой Бутстреп мы нашли среднюю прибыль,  95%-й доверительный интервал и риск убытков для каждого региона

По условию после оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирать регион с наибольшей средней прибылью. Но из данных регионов только один регион имеет риск меньше 2.5% 

В разработку мы берём Регион 1 - Риск 0.3%, у остальных регионов риски намного больше и нам они не подходят, поэтому нет необходимости сравнивать среднюю прибыль, хотя она также выше в этом регионе

Средняя прибыль: 420194005.3440501 рублей


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован